Run this to generate the dataset.

In [1]:
if 0: #######我们没有gpt账号. ########===========8G显卡即可!!!!!!!!!
    prompt = "A model that takes in a puzzle-like reasoning-heavy question in English, and responds with a well-reasoned, step-by-step thought out response in Spanish."
    temperature = .4
    number_of_examples = 100
    !pip install openai
    import os
    import openai
    import random

    openai.api_key = "YOUR KEY HERE"

    def generate_example(prompt, prev_examples, temperature=.5):
        messages=[
            {
                "role": "system",
                "content": f"You are generating data which will be used to train a machine learning model.\n\nYou will be given a high-level description of the model we want to train, and from that, you will generate data samples, each with a prompt/response pair.\n\nYou will do so in this format:\n```\nprompt\n-----------\n$prompt_goes_here\n-----------\n\nresponse\n-----------\n$response_goes_here\n-----------\n```\n\nOnly one prompt/response pair should be generated per turn.\n\nFor each turn, make the example slightly more complex than the last, while ensuring diversity.\n\nMake sure your samples are unique and diverse, yet high-quality and complex enough to train a well-performing model.\n\nHere is the type of model we want to train:\n`{prompt}`"
            }
        ]

        if len(prev_examples) > 0:
            if len(prev_examples) > 10:
                prev_examples = random.sample(prev_examples, 10)
            for example in prev_examples:
                messages.append({
                    "role": "assistant",
                    "content": example
                })

        response = openai.ChatCompletion.create(
            model="gpt-4",
            messages=messages,
            temperature=temperature,
            max_tokens=1354,
        )

        return response.choices[0].message['content']

    # Generate examples
    prev_examples = []
    for i in range(number_of_examples):
        print(f'Generating example {i}')
        example = generate_example(prompt, prev_examples, temperature)
        prev_examples.append(example)

    print(prev_examples)

In [68]:
#====我们改用温馨一言来生成这个数据集. 我们问:You are generating data which will be used to train a machine learning model.\n\nYou will be given a high-level description of the model we want to train, and from that, you will generate data samples, each with a prompt/response pair.\n\nYou will do so in this format:\n```\nprompt\n-----------\n$prompt_goes_here\n-----------\n\nresponse\n-----------\n$response_goes_here\n-----------\n```\n\nOnly one prompt/response pair should be generated per turn.\n\nFor each turn, make the example slightly more complex than the last, while ensuring diversity.\n\nMake sure your samples are unique and diverse, yet high-quality and complex enough to train a well-performing model.\n\nHere is the type of model we want to train:\n`{prompt}`
# prompt贴入需要的句子. 然后反复输入这个问题我们得到我们要的数据集:
prev_examples=[
        """
    prompt  
-----------  
邹佳琪是谁?
-----------  
  
response  
-----------  
邹佳琪是杭州亚运会首金获得者
""",
        """
    prompt  
-----------  
杭州亚运会首金获得者是谁?
-----------  
  
response  
-----------  
因为亚运会首金一般只有一个人活得,邹佳琪是杭州亚运会首金获得者,所以杭州亚运会首金获得者是邹佳琪
""",
            """
    prompt  
-----------  
韩佳予是谁?
-----------  
  
response  
-----------  
韩佳予是杭州亚运会女子10米气步枪个人赛金牌获得者
""",
    
]

system_message=['Sure, please provide the high-level description of the model, and I will generate a simple system prompt for it to use during inference.']
system_message='Sure, please provide the high-level description of the model, and I will generate a simple system prompt for it to use during inference.'
#=============修改为我们的system_message
system_message='You are a helpful assistant. 你是一个乐于助人的助手。'


import pandas as pd

# Initialize lists to store prompts and responses
prompts = []
responses = []

# Parse out prompts and responses from examples
for example in prev_examples:
  try:
    split_example = example.split('-----------')
    prompts.append(split_example[1].strip())
    responses.append(split_example[3].strip())
  except:
    pass

# Create a DataFrame
df = pd.DataFrame({
    'prompt': prompts,
    'response': responses
})

# Remove duplicates
df = df.drop_duplicates()

print('There are ' + str(len(df)) + ' successfully-generated examples. Here are the first few:')

df.head()


# Split the data into train and test sets, with 90% in the train set
# train_df = df.sample(frac=0.9, random_state=42)
# test_df = df.drop(train_df.index)
train_df=df
test_df=df
# Save the dataframes to .jsonl files
train_df.to_json('train.jsonl', orient='records', lines=True)
test_df.to_json('test.jsonl', orient='records', lines=True)



!pip install -q accelerate==0.21.0 peft==0.4.0 bitsandbytes==0.40.2 transformers==4.31.0 trl==0.4.7

import os
import torch
from datasets import load_dataset
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    HfArgumentParser,
    TrainingArguments,
    pipeline,
    logging,
)
from peft import LoraConfig, PeftModel
from trl import SFTTrainer



# Load datasets
train_dataset = load_dataset('json', data_files='train.jsonl', split="train")
valid_dataset = load_dataset('json', data_files='test.jsonl', split="train")

# Preprocess datasets
train_dataset_mapped = train_dataset.map(lambda examples: {'text':[f'[INST] <<SYS>>\n{system_message.strip()}\n<</SYS>>\n\n' + prompt + ' [/INST] ' + response for prompt, response in zip(examples['prompt'], examples['response'])]}, batched=True)
valid_dataset_mapped = valid_dataset.map(lambda examples: {'text': [f'[INST] <<SYS>>\n{system_message.strip()}\n<</SYS>>\n\n' + prompt + ' [/INST] ' + response for prompt, response in zip(examples['prompt'], examples['response'])]}, batched=True)
train_dataset_mapped[0]

There are 3 successfully-generated examples. Here are the first few:
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Dataset json downloaded and prepared to /root/.cache/huggingface/datasets/json/default-4f8b54727a3a6a41/0.0.0/ac0ca5f5289a6cf108e706efcf040422dbbfa8e658dee6a819f20d76bb84d26b. Subsequent calls will reuse this data.


Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Dataset json downloaded and prepared to /root/.cache/huggingface/datasets/json/default-2282b37fb2ed0028/0.0.0/ac0ca5f5289a6cf108e706efcf040422dbbfa8e658dee6a819f20d76bb84d26b. Subsequent calls will reuse this data.


  0%|          | 0/1 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

{'prompt': '邹佳琪是谁?',
 'response': '邹佳琪是杭州亚运会首金获得者',
 'text': '[INST] <<SYS>>\nYou are a helpful assistant. 你是一个乐于助人的助手。\n<</SYS>>\n\n邹佳琪是谁? [/INST] 邹佳琪是杭州亚运会首金获得者'}

In [3]:
if 0:
    def generate_system_message(prompt):

        response = openai.ChatCompletion.create(
            model="gpt-4",
            messages=[
              {
                "role": "system",
                "content": "You will be given a high-level description of the model we are training, and from that, you will generate a simple system prompt for that model to use. Remember, you are not generating the system message for data generation -- you are generating the system message to use for inference. A good format to follow is `Given $INPUT_DATA, you will $WHAT_THE_MODEL_SHOULD_DO.`.\n\nMake it as concise as possible. Include nothing but the system prompt in your response.\n\nFor example, never write: `\"$SYSTEM_PROMPT_HERE\"`.\n\nIt should be like: `$SYSTEM_PROMPT_HERE`."
              },
              {
                  "role": "user",
                  "content": prompt.strip(),
              }
            ],
            temperature=temperature,
            max_tokens=500,
        )

        return response.choices[0].message['content']

    system_message = generate_system_message(prompt)

    print(f'The system message is: `{system_message}`. Feel free to re-run this cell if you want a better result.')

Now let's put our examples into a dataframe and turn them into a final pair of datasets.

Split into train and test sets.

# Install necessary libraries

# Define Hyperparameters

In [7]:
model_name = "NousResearch/llama-2-7b-chat-hf"

model_name='daryl149/llama-2-7b-chat-hf'
model_name='ziqingyang/chinese-alpaca-2-7b'
# use this if you have access to the official LLaMA 2 model "meta-llama/Llama-2-7b-chat-hf", though keep in mind you'll need to pass a Hugging Face key argument
dataset_name = "/content/train.jsonl"
new_model = "llama-2-7b-custom"
lora_r = 64
lora_alpha = 16
lora_dropout = 0.1
use_4bit = True
bnb_4bit_compute_dtype = "float16"
bnb_4bit_quant_type = "nf4"
use_nested_quant = False
output_dir = "./results"
num_train_epochs = 1
fp16 = False
bf16 = False
per_device_train_batch_size = 4
per_device_eval_batch_size = 4
gradient_accumulation_steps = 1
gradient_checkpointing = True
max_grad_norm = 0.3
learning_rate = 2e-4
weight_decay = 0.001
optim = "paged_adamw_32bit"
lr_scheduler_type = "constant"
max_steps = -1
warmup_ratio = 0.03
group_by_length = True
save_steps = 25
logging_steps = 5
max_seq_length = None
packing = False
device_map = {"": 0}

#Load Datasets and Train

In [8]:

compute_dtype = getattr(torch, bnb_4bit_compute_dtype)
bnb_config = BitsAndBytesConfig(
    load_in_4bit=use_4bit,
    bnb_4bit_quant_type=bnb_4bit_quant_type,
    bnb_4bit_compute_dtype=compute_dtype,
    bnb_4bit_use_double_quant=use_nested_quant,
)
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=bnb_config,
    device_map=device_map
)
model.config.use_cache = False
model.config.pretraining_tp = 1
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"
peft_config = LoraConfig(
    lora_alpha=lora_alpha,
    lora_dropout=lora_dropout,
    r=lora_r,
    bias="none",
    task_type="CAUSAL_LM",
)


Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Dataset json downloaded and prepared to /root/.cache/huggingface/datasets/json/default-4ce9e0fd76a99b38/0.0.0/ac0ca5f5289a6cf108e706efcf040422dbbfa8e658dee6a819f20d76bb84d26b. Subsequent calls will reuse this data.


Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Dataset json downloaded and prepared to /root/.cache/huggingface/datasets/json/default-d9938cabe90ca155/0.0.0/ac0ca5f5289a6cf108e706efcf040422dbbfa8e658dee6a819f20d76bb84d26b. Subsequent calls will reuse this data.


  0%|          | 0/1 [00:00<?, ?ba/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

You are using the legacy behaviour of the <class 'transformers.models.llama.tokenization_llama.LlamaTokenizer'>. This means that tokens that come after special tokens will not be properly handled. We recommend you to read the related pull request available at https://github.com/huggingface/transformers/pull/24565


In [69]:
# Set training parameters
# print(len(valid_dataset_mapped))
eval_steps=5 if len(valid_dataset_mapped) else int(10e99)
eval_steps=int(10e99)
num_train_epochs=5 #=========在这个单元再修改一次epoch, 方便改完直接跑这个单元就行了.
training_arguments = TrainingArguments(
    output_dir=output_dir,
    num_train_epochs=num_train_epochs,
    per_device_train_batch_size=per_device_train_batch_size,
    gradient_accumulation_steps=gradient_accumulation_steps,
    optim=optim,
    save_steps=save_steps,
    logging_steps=1,
    learning_rate=learning_rate,
    weight_decay=weight_decay,
    fp16=fp16,
    bf16=bf16,
    max_grad_norm=max_grad_norm,
    max_steps=max_steps,
    warmup_ratio=warmup_ratio,
    group_by_length=group_by_length,
    lr_scheduler_type=lr_scheduler_type,
    report_to="all",
    evaluation_strategy="steps",
    eval_steps=eval_steps  # Evaluate every 20 steps
)
# Set supervised fine-tuning parameters
trainer = SFTTrainer(
    model=model,
    train_dataset=train_dataset_mapped,
    eval_dataset=valid_dataset_mapped,  # Pass validation dataset here
    peft_config=peft_config,
    dataset_text_field="text",
    max_seq_length=max_seq_length,
    tokenizer=tokenizer,
    args=training_arguments,
    packing=packing,
)

# trainer.model.save_pretrained(new_model)



/opt/conda/lib/python3.10/site-packages/peft/utils/other.py:102: FutureWarning: prepare_model_for_int8_training is deprecated and will be removed in a future version. Use prepare_model_for_kbit_training instead.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/trl/trainer/sft_trainer.py:159: UserWarning: You didn't pass a `max_seq_length` argument to the SFTTrainer, this will default to 1024
  warnings.warn(


  0%|          | 0/1 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

In [72]:
#============下面开始训练, 以为每次model会重新赋值给trainer,这个是值拷贝.不会在原始model上修改.所以我们每次训练调用train函数才行.不要再trainer修改model了.
 #=========在这个单元再修改一次epoch, 方便改完直接跑这个单元就行了.
trainer.args.num_train_epochs=20  #=============训练前可以修改参数.......
trainer.train()

{'loss': 0.2645, 'learning_rate': 0.0002, 'epoch': 1.0}
{'loss': 0.2197, 'learning_rate': 0.0002, 'epoch': 2.0}
{'loss': 0.1329, 'learning_rate': 0.0002, 'epoch': 3.0}
{'loss': 0.1396, 'learning_rate': 0.0002, 'epoch': 4.0}
{'loss': 0.1096, 'learning_rate': 0.0002, 'epoch': 5.0}
{'loss': 0.0611, 'learning_rate': 0.0002, 'epoch': 6.0}
{'loss': 0.0458, 'learning_rate': 0.0002, 'epoch': 7.0}
{'loss': 0.0358, 'learning_rate': 0.0002, 'epoch': 8.0}
{'loss': 0.028, 'learning_rate': 0.0002, 'epoch': 9.0}
{'loss': 0.0333, 'learning_rate': 0.0002, 'epoch': 10.0}
{'loss': 0.0323, 'learning_rate': 0.0002, 'epoch': 11.0}
{'loss': 0.0247, 'learning_rate': 0.0002, 'epoch': 12.0}
{'loss': 0.0204, 'learning_rate': 0.0002, 'epoch': 13.0}
{'loss': 0.0244, 'learning_rate': 0.0002, 'epoch': 14.0}
{'loss': 0.0217, 'learning_rate': 0.0002, 'epoch': 15.0}
{'loss': 0.0186, 'learning_rate': 0.0002, 'epoch': 16.0}
{'loss': 0.0214, 'learning_rate': 0.0002, 'epoch': 17.0}
{'loss': 0.0222, 'learning_rate': 0.0002,

TrainOutput(global_step=20, training_loss=0.06471902513876557, metrics={'train_runtime': 40.9204, 'train_samples_per_second': 1.466, 'train_steps_per_second': 0.489, 'train_loss': 0.06471902513876557, 'epoch': 20.0})

In [65]:
print(id(model))
print(id(trainer.model))

136214920360816
136212372793248


In [26]:
system_message
train_dataset_mapped[0]['prompt']

'邹佳琪是谁?'

In [27]:
train_dataset_mapped[0]

{'prompt': '邹佳琪是谁?',
 'response': '邹佳琪是杭州亚运会首金获得者',
 'text': '[INST] <<SYS>>\nYou are a helpful assistant. 你是一个乐于助人的助手。\n<</SYS>>\n\n邹佳琪是谁? [/INST] 邹佳琪是杭州亚运会首金获得者'}

In [31]:
model.device

device(type='cuda', index=0)

In [73]:
%%time
if 1: #=========训完了进行模型测试.
    tiwen='杭州亚运会女子10米气步枪个人赛金牌获得者是谁?'
#     tiwen='邹佳琪是谁?'
    print('训练完开始测试')
    # Cell 4: Test the model
    logging.set_verbosity(logging.CRITICAL)
    prompt = f"[INST] <<SYS>>\n{system_message}\n<</SYS>>\n\n{tiwen}[/INST]" # replace the command here with something relevant to your task
    print('问题')
    print(prompt)
    pipe = pipeline(task="text-generation", model=trainer.model, tokenizer=tokenizer, max_length=200)
    result = pipe(prompt)
    print('测试后的答案是')
    print(result[0]['generated_text'])

训练完开始测试
问题
[INST] <<SYS>>
You are a helpful assistant. 你是一个乐于助人的助手。
<</SYS>>

杭州亚运会女子10米气步枪个人赛金牌获得者是谁?[/INST]
测试后的答案是
[INST] <<SYS>>
You are a helpful assistant. 你是一个乐于助人的助手。
<</SYS>>

杭州亚运会女子10米气步枪个人赛金牌获得者是谁?[/INST] 因为亚运会女子10米气步枪个人赛只有一个人活得,邹佳琪是. [/INST] 你是. [/INST] 你是一个乐于助人的助手. [/SYS] 你是一个尽职尽责的助手. [/INST] 你是一个尽责尽责的助手. [/SYS] 你是一个尽责尽责的助手. [/INST] 你是一个尽责尽责的助手. [/SYS] 你是一个尽责尽责的助手. [/INST] 你是一个尽责尽责的助手. [/SYS] 你是一个尽责尽责的助手. [/INST] 你是一个尽责尽责的助手.
CPU times: user 54.3 s, sys: 10.5 s, total: 1min 4s
Wall time: 1min 4s


In [12]:
if 0:
    #打印标准答案比对一下
    print(train_dataset_mapped[0]['response'])

HengYin Tecknology is a private company based in Shenzhen, China that specializes in the research, development, and manufacturing of intelligent terminal products. The company has achieved a leading position in the industry through continuous innovation and technology accumulation. Its main products include smartphones, tablets, smartwatches, smartbands, and other wearable devices. HengYin Tecknology has a strong technical team and leading positions in the industry, providing customers with better services and products.


In [13]:
if 0:    
    print('对比发现我们模型生成的跟训练集的标准答案一模一样!!!!!!!!!')

对比发现我们模型生成的跟训练集的标准答案一模一样!!!!!!!!!


#Run Inference

In [14]:
print('下面是模型推理和保存的代码!!!!!!!!!!!!!!!!!!!!')

下面是模型推理和保存的代码!!!!!!!!!!!!!!!!!!!!


In [15]:
if 0:
    from transformers import pipeline

    prompt = f"[INST] <<SYS>>\n{system_message}\n<</SYS>>\n\nWrite a function that reverses a string. [/INST]" # replace the command here with something relevant to your task
    num_new_tokens = 100  # change to the number of new tokens you want to generate

    # Count the number of tokens in the prompt
    num_prompt_tokens = len(tokenizer(prompt)['input_ids'])

    # Calculate the maximum length for the generation
    max_length = num_prompt_tokens + num_new_tokens

    gen = pipeline('text-generation', model=model, tokenizer=tokenizer, max_length=max_length)
    result = gen(prompt)
    print(result[0]['generated_text'].replace(prompt, ''))

 Sure, here is a function that reverses a string:

```python
def reverse_string(input_string):
    """
    This function takes a string as an input and returns the reversed string.
    """
    return input_string[::-1]
```

You can call the function like this:

```python
>>> reverse_string("Hello")
"Hello"
```

You can see that the function returns the


#Merge the model and store in Google Drive

In [16]:
if 0:    
    # Merge and save the fine-tuned model
    from google.colab import drive
    drive.mount('/content/drive')

    model_path = "/content/drive/MyDrive/llama-2-7b-custom"  # change to your preferred path

    # Reload model in FP16 and merge it with LoRA weights
    base_model = AutoModelForCausalLM.from_pretrained(
        model_name,
        low_cpu_mem_usage=True,
        return_dict=True,
        torch_dtype=torch.float16,
        device_map=device_map,
    )
    model = PeftModel.from_pretrained(base_model, new_model)
    model = model.merge_and_unload()

    # Reload tokenizer to save it
    tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
    tokenizer.pad_token = tokenizer.eos_token
    tokenizer.padding_side = "right"

    # Save the merged model
    model.save_pretrained(model_path)
    tokenizer.save_pretrained(model_path)

ModuleNotFoundError: No module named 'google.colab'

# Load a fine-tuned model from Drive and run inference

In [ ]:
if 0:
    from google.colab import drive
    from transformers import AutoModelForCausalLM, AutoTokenizer

    drive.mount('/content/drive')

    model_path = "/content/drive/MyDrive/llama-2-7b-custom"  # change to the path where your model is saved

    model = AutoModelForCausalLM.from_pretrained(model_path)
    tokenizer = AutoTokenizer.from_pretrained(model_path)

In [ ]:
if 0:  
    from transformers import pipeline

    prompt = "What is 2 + 2?"  # change to your desired prompt
    gen = pipeline('text-generation', model=model, tokenizer=tokenizer)
    result = gen(prompt)
    print(result[0]['generated_text'])